In [29]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/itv007008/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [30]:
spark

In [3]:
words = ("big", "Data", "IS", "Super", "interesting", "BIG", "DATA", "IS", "TRENDING", "technology")

In [4]:
words_rdd = spark.sparkContext.parallelize(words)

In [5]:
words_normalized = words_rdd.map(lambda x: x.lower()).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).sortByKey()

In [6]:
##How to check the number of partitions
words_normalized.getNumPartitions()

2

In [7]:
spark.sparkContext.defaultParallelism

2

In [8]:
readFileRdd = spark.sparkContext.textFile('/public/trendytech/retail_db/orders/*')

In [9]:
readFileRdd.getNumPartitions()

2

In [10]:
spark.sparkContext.defaultMinPartitions

2

In [11]:
big_log_rdd = spark.sparkContext.textFile('/public/trendytech/bigLog.txt')

In [12]:
big_log_rdd.getNumPartitions()

3

In [13]:
big_log_rdd.take(2)

['ERROR: Thu Jun 04 10:37:51 BST 2015', 'WARN: Sun Nov 06 10:37:51 GMT 2016']

In [14]:
mapped_rdd=readFileRdd.map(lambda x: (x.split(',')[3],1)).reduceByKey(lambda x,y: x+y)

In [15]:
mapped_rdd.collect()

[('CLOSED', 7556),
 ('CANCELED', 1428),
 ('PENDING_PAYMENT', 15030),
 ('COMPLETE', 22899),
 ('PROCESSING', 8275),
 ('PAYMENT_REVIEW', 729),
 ('PENDING', 7610),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558)]

In [16]:
count_by_value = readFileRdd.map(lambda x: (x.split(',')[3]))

In [17]:
count_by_value.countByValue()

defaultdict(int,
            {'CLOSED': 7556,
             'PENDING_PAYMENT': 15030,
             'COMPLETE': 22899,
             'PROCESSING': 8275,
             'PAYMENT_REVIEW': 729,
             'PENDING': 7610,
             'ON_HOLD': 3798,
             'CANCELED': 1428,
             'SUSPECTED_FRAUD': 1558})

In [18]:
### Reduce Vs ReduceByKey

my_list = [1,3,5,6,7,8]

In [19]:
parallel_rdd = spark.sparkContext.parallelize(my_list)

In [20]:
### Reduce is Action ####
parallel_rdd.reduce(lambda x,y: x+y)

30

In [31]:
### ReduceByKey vs GroupByKey

base_rdd = spark.sparkContext.textFile('/public/trendytech/orders/orders.csv').map(lambda x: (x.split(',')[3],1)).reduceByKey(lambda x,y: x+y)

In [32]:
base_rdd.collect()

[('PENDING', 9512500),
 ('PROCESSING', 10343750),
 ('CANCELED', 1785000),
 ('COMPLETE', 28623750),
 ('SUSPECTED_FRAUD', 1947500),
 ('CLOSED', 9445000),
 ('ON_HOLD', 4747500),
 ('PAYMENT_REVIEW', 911250),
 ('PENDING_PAYMENT', 18787500)]

In [33]:
spark

In [37]:
groupByKeyRdd = spark.sparkContext.textFile('/public/trendytech/orders/orders.csv').map(lambda x: (x.split(',')[3],x.split(',')[2]))

In [38]:
groupByKeyRdd.take(3)

[('CLOSED', '11599'), ('PENDING_PAYMENT', '256'), ('COMPLETE', '12111')]

In [45]:
spark

In [44]:
result = groupByKeyRdd.map(lambda x: (x[0], len(x[1])))